<a href="https://colab.research.google.com/github/josephhchoi/Bellevue-DSC/blob/main/DSC410_W11_Dash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DSC410: Dash

## Table of Content:
- Model Setup
- Dash App
- Additional Documentation

## Model Setup:
- Setting up my scaler and random forest classifier model from week 8

In [9]:
# Setup
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.preprocessing import StandardScaler

In [1]:
# Loading in the cleaned, preprocessed, and feature engineered df from W8 exercise
travel_insurance_final = pd.read_csv('/content/travel_insurance_final.csv')

# Splitting the data into features and target variable
X = travel_insurance_final.drop('Claim', axis=1)
y = travel_insurance_final['Claim']

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Splitting the data into features and target variable
X = travel_insurance_final.drop('Claim', axis=1)
y = travel_insurance_final['Claim']

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardizing the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Defining the random forest classifier
rf_classifier = RandomForestClassifier()

# Fitting the model to the training data
rf_classifier.fit(X_train_scaled, y_train)

# Making predictions on the test set
y_pred = rf_classifier.predict(X_test_scaled)

## Dash App:
- Implementing my scaler and random forest classifier model into Dash
- Referenced code in textbook Hands-on Predictive Analytics (pg. 292-300)

In [10]:
# Setup
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output, State
import joblib
import numpy as np
import pandas as pd

In [7]:
"""
Code Description:
  - Serializing and saving machine learning models via "joblib"
  - Dealing with two objects:
    - RandomForestClassifier model
    - StandardScaler object

Code Breakdown:
  - Step 1: Specifying file paths
  - Step 2: Saving the "rf_classifier" model
  - Step 3: Saving the "scaler" object

"""

# Step 1:
model_path = '/content/rf_classifier.pkl'
scaler_path = '/content/scaler.pkl'

# Step 2:
joblib.dump(rf_classifier, model_path)

# Step 3:
joblib.dump(scaler, scaler_path)

['/content/scaler.pkl']

In [8]:
"""
Code Description:
  - Coding for Dash app initialization and the configuration of the web application
  - Setting up the app's layout with:
    - Input fields
    - Predict button to start the prediction process

Code Breakdown:
  - Step 1: Initializing Dash app
  - Step 2: Defining the app layout
  - App layout is sectioned off into 4 parts:
    - Step 3 (Title): Displaying main header of the app "Travel Insurance Claim Prediction" via "html.H1()"
    - Step 4 (Inputs): Creating an input area where users can fill out info for each field via "dcc.Input()"
    - Step 5 (Predict Button): Creating a button that triggers prediction when data is updated via "html.Button()"
    - Step 6 (Output Display): Allowing it to show prediction results when button is triggered via "html.Div()"
"""

# Step 1:
app = dash.Dash(__name__)
server = app.server

# Step 2:
app.layout = html.Div([
    # Step 3:
    html.H1("Travel Insurance Claim Prediction"),
    # Step 4:
    html.Div([
        html.Div([
            html.Label("Agency:"),
            dcc.Input(id="agency", type="number", value=1),
        ], className="three columns"),
        html.Div([
            html.Label("Product Name:"),
            dcc.Input(id="product_name", type="number", value=1),
        ], className="three columns"),
        html.Div([
            html.Label("Duration:"),
            dcc.Input(id="duration", type="number", value=186),
        ], className="three columns"),
        html.Div([
            html.Label("Destination:"),
            dcc.Input(id="destination", type="number", value=1),
        ], className="three columns"),
        html.Div([
            html.Label("Net Sales:"),
            dcc.Input(id="net_sales", type="number", value=-29.0),
        ], className="three columns"),
        html.Div([
            html.Label("Commission (in value):"),
            dcc.Input(id="commission", type="number", value=9.57),
        ], className="three columns"),
        html.Div([
            html.Label("Age:"),
            dcc.Input(id="age", type="number", value=81),
        ], className="three columns"),
        html.Div([
            html.Label("Agency Type Encoded:"),
            dcc.Input(id="agency_type_encoded", type="number", value=0),
        ], className="three columns"),
        html.Div([
            html.Label("Distribution Channel Encoded:"),
            dcc.Input(id="distribution_channel_encoded", type="number", value=1),
        ], className="three columns"),
    ], className="row"),
    # Step 5:
    html.Button("Predict", id="predict-btn", n_clicks=0),
    # Step 6:
    html.Div(id="prediction-output", style={"marginTop": "20px"}),
])

"""
Code Description:
  - Implementing callback function to handle predictions in Dash
  - Setting it up so that it updates the output display based on user inputs

Code Breakdown:
  - Step 7: Setting up Callback via "@app.callback"
    - Linking the predict button's clicks to the prediction output
    - Defining inputs and states to capture user inputs from the app's interface
      - Input:
      - Output:
      - State:
  - Step 8: Creating the "predict" function to process inputs and generate predictions:
    - Step 9: Compiling all the input values into array format via "np.array()"
    - Step 10: Applying scaling to the input features with "scaler" object
    - Step 11: Implementing the "rf_classifier" model to make predictions
    - Step 12: Returning prediction result. Features:
      - Displaying "The model predicts:" output ("Claim/No Claim") based on the outcome
      - Prompting the user to "Enter features and click predict" if no prediction is made
  - Step 13: Running the App
    - "app.run_server(debug=True)": Making the app available for interaction.
"""

# Step 7:
@app.callback(
    Output("prediction-output", "children"),
    [Input("predict-btn", "n_clicks")],
    [State("agency", "value"), State("product_name", "value"),
     State("duration", "value"), State("destination", "value"),
     State("net_sales", "value"), State("commission", "value"),
     State("age", "value"), State("agency_type_encoded", "value"),
     State("distribution_channel_encoded", "value")]
)
# Step 8:
def predict(n_clicks, agency, product_name, duration, destination, net_sales, commission, age, agency_type_encoded, distribution_channel_encoded):
    if n_clicks > 0:
        # Step 9:
        input_features = np.array([[agency, product_name, duration, destination, net_sales, commission, age, agency_type_encoded, distribution_channel_encoded]])

        # Step 10:
        input_features_scaled = scaler.transform(input_features)

        # Step 11:
        prediction = rf_classifier.predict(input_features_scaled)

        # Step 12:
        if prediction[0] == 1:
            return "The model predicts: Claim"
        else:
            return "The model predicts: No Claim"
    return "Enter features and click predict"

# Step 13:
if __name__ == '__main__':
    app.run_server(debug=True)

<IPython.core.display.Javascript object>

## Additional Documentation:

For this week's exercise, my initial working environment was Docker and through Docker, I wrote my codes within a Jupyter notebook. However, I was unable to launch the Dash app server in the Docker environment. This issue occurred for both my w11 exercise and the diamond example code.

Following your suggestion, I transitioned over to Google Colab and everything worked! I am also including a screenshot of my launced Dash app in case you are unable to see the app's output.